In [21]:
import os
import argparse
import soundfile as sf
import pandas as pd
import numpy as np
import functools
from scipy.signal import resample_poly
import tqdm.contrib.concurrent
import random

def get_list_from_csv(row, column, n_src):
    """ Transform a list in the .csv in an actual python list """
    python_list = []
    for i in range(n_src):
        current_column = column.split('_')
        current_column.insert(1, str(i + 1))
        current_column = '_'.join(current_column)
        python_list.append(row[current_column])
    return python_list


md_file = pd.read_csv("metadata/LibriMix/librimix_test-clean.csv", engine='python')
row = md_file.iloc[0]
sources_path_list = get_list_from_csv(row, 'source_path', 3)
gain_list = get_list_from_csv(row, 'source_gain', 3)
sources_list = []
for sources_path in sources_path_list:
    source, _ = sf.read(sources_path, dtype='float32')
    sources_list.append(source)


noise_path = row['noise_path']
noise, _ = sf.read(noise_path, dtype='float32', start=row['noise_l_endpoint'], stop=row['noise_r_endpoint'])
if len(noise.shape) > 1:   noise = noise[:, 0]

sources_list.append(noise)
gain_list.append(row['noise_gain'])

print(row["SNR"])

from create_dataset_from_metadata import transform_sources, mix
transformed_sources = transform_sources(sources_list, 16000, 'min', gain_list)

# print(len(transformed_sources))
# mixture = mix(transformed_sources, normalize=True)

mixture = np.zeros_like(transformed_sources[0])
length = len(mixture)

step = 1600
st = 0
while st + step < length:
    choice = np.random.choice([0,1,2,3], p=[1/8, 2/8, 2/8, 3/8])
    print(choice)
    mixture[st:st+step] = transformed_sources[-1][st:st+step]
    choose = random.sample([0,1,2], choice)
    for id in choose:
        mixture[st:st+step] += transformed_sources[id][st:st+step]
    st += step
print(mixture)


sf.write("temp.wav", mixture, 16000)

6.513568451402625
1
2
3
3
2
2
3
3
1
2
1
2
1
0
3
3
0
0
3
3
3
3
3
3
1
3
3
1
3
3
0
3
1
0
3
1
1
[0.03483292 0.02396084 0.01565401 ... 0.         0.         0.        ]


In [5]:
import soundfile as sf
import numpy as np

p= np.load("/home/getsum/code/temp/data/LibriMix/wav16k/min/test/label/0dB_61-70968-0003_7021-79740-0011_4446-2275-0012_label.npy")
print(p)

[2 2 3]
